In [ ]:
%pip install trame
%pip install vtk
%pip install fiona
%pip install gemgis
%pip install rasterio
%pip install pyvista
%pip install trimesh
%pip install pyglet
%pip install folium

In [ ]:
import numpy as np
import trimesh
import re

In [ ]:
def read_ts(tsfile):
    vrtx = []
    trgl = []
    #split atom line by by space, make vertex at that point in VRTX list
    #go back and get atomlist[1]  VRTXLIST[atomlist[1]-1]

    for line in tsfile:
        if 'VRTX' in line:
            l_input = re.split('[\s]+', line)
            temp = np.array(l_input[2:5])
            #temp = np.array(l_input[2:6])
            vrtx.append(temp.astype(float))

        if 'ATOM' in line:
            l_input = re.split('[\s*]', line)
            vertex_id = l_input[1]
            vertex_id_atom = l_input[2]
            #print(len(vrtx), vertex_id, vertex_id_atom)
            vrtx.append(vrtx[ int(vertex_id_atom) -1])

        if 'TRGL' in line:
            l_input = re.split('[\s*]', line)
            temp = np.array(l_input[1:4])
            trgl.append(temp.astype(int))

    vrtx = np.asarray(vrtx)
    trgl = np.asarray(trgl)

    crs_dict = {"NAME":None,"AXIS_NAME":None,"AXIS_UNIT":None,"ZPOSITIVE":None}
    check_dist = {"SURFACE":tsfile[-1],"NAME":None,"COORD":0,"TFACE":0,"PVRTX":0,"VRTX":0,"TRGL":0,"ATOM":0,"CRS":crs_dict, "COLOR":None}

    for line in tsfile:
        ##HEADER
        if 'color:' in line:
            strcolor = line.split(':')
            tricolor = re.split('[\s*]', strcolor[1])
            tricolor.pop()
            tricolor[0] = float(tricolor[0]) #rgb 0-1 colors R
            tricolor[1] = float(tricolor[1]) #rgb 0-1 colors G
            tricolor[2] = float(tricolor[2]) #rgb 0-1 colors B
            check_dist["COLOR"] = tricolor
        if 'name:' in line:
            strname = line.split(':')
            usename = strname[1].replace("\n",'')
            check_dist["NAME"] = usename
        if 'NAME ' in line and "AXIS" not in line:
            strname = re.split('[\s*]', line)
            crs = strname[1].replace("\n",'')
        if 'AXIS_NAME' in line:
            axis_name = re.split('[\s*]', line)
            axis_name.pop()
        if 'AXIS_UNIT' in line:
            axis_unit = re.split('[\s*]', line)
            axis_unit.pop()
        if 'ZPOSITIVE' in line:
            zpositive = re.split('[\s*]', line)[1].replace("\n",'')
        if 'END_ORIGINAL_COORDINATE' in line:
            crs_dict["NAME"] = crs
            crs_dict["AXIS_NAME"] = axis_name
            crs_dict["AXIS_UNIT"] = axis_unit
            crs_dict["ZPOSITIVE"] = zpositive

        ####Data
        if 'TFACE' in line:
            check_dist["TFACE"] += 1
        if 'COORDINATE_SYSTEM' in line:
            check_dist["COORD"] += 1
        if 'ATOM' in line:
            check_dist["ATOM"] += 1
        if 'PVRTX' in line:
            check_dist["PVRTX"] += 1
        if 'VRTX' in line:
            check_dist["VRTX"] += 1
        if 'TRGL' in line:
            check_dist["TRGL"] += 1

    return vrtx, trgl, check_dist

In [ ]:
#Get anomalyName from input
from IPython.display import display, Markdown
import ipywidgets as widgets
anomalyName_text = widgets.Text(value='Pilbara',
                placeholder='Copy or paste the anomalyName_text.',
                description='Name:')
display(Markdown('## Enter AnomalyName for search.'))
display(anomalyName_text)

In [ ]:
#Searching
from io import StringIO
import pandas as pd
from auscopecat.network import request

name = f'{anomalyName_text.value}' #'Pilbara'
print(f'searching for {name}')
cql_filter = f'AnomalyName like \'%{name}%\''
url = 'https://remanentanomalies.csiro.au/geoserver/ows'
params = {
            'service': 'WFS',
            'version': '1.1.0',
            'request': 'GetFeature',
            'typename': 'RemAnom:Anomaly',
            'outputFormat': 'csv',
            'srsname': 'EPSG:4326',
            'CQL_FILTER': cql_filter,
            'maxFeatures': str(10)
            }
try:
    response = request(url,params,'POST')
except Exception as e:
    print(f'Error querying data: {e}',500)
csvBuffer = StringIO(response.text)
df = pd.read_csv(filepath_or_buffer = csvBuffer, low_memory=False)
print(df.shape)
df.head(10)
    

In [ ]:
#import all libs for anomalies
import pandas as pd
import folium
from folium.plugins import BeautifyIcon
from branca.element import Figure

pd.options.mode.chained_assignment = None
pd.set_option('display.width', 400)

#show anomalies on map
fig=Figure(width=1024,height=768)
m=folium.Map(location=[-23,130],zoom_start=4)
fig.add_child(m)
# Add custom base maps to folium
basemaps = {
    'Google Maps': folium.TileLayer(tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',attr = 'Google',name = 'Google Maps',overlay = False,control = True),
    'Google Satellite': folium.TileLayer(tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',attr = 'Google',name = 'Google Satellite',overlay = False,control = True),
    'Google Terrain': folium.TileLayer(tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',attr = 'Google',name = 'Google Terrain',overlay = False,control = True),
    'Google Satellite Hybrid': folium.TileLayer(tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',attr = 'Google',name = 'Google Satellite',overlay = False,control = True),
    'Esri Satellite': folium.TileLayer(tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',attr = 'Esri',name = 'Esri Satellite',overlay = False,control = True)
}
basemaps['Google Maps'].add_to(m)
basemaps['Google Satellite Hybrid'].add_to(m)
basemaps['Google Satellite'].add_to(m)
basemaps['Google Terrain'].add_to(m)
basemaps['Esri Satellite'].add_to(m)
folium.LayerControl().add_to(m)
#add circle-dot for anomalies boreholes:
for row in df.itertuples():
  print(f'{row[1]}  {row[7]}  {row[29]}')
  point = row[29]
  found = re.search(r'\((.+?)\)', point)
  if not found:
    continue
  cx, cy = found.group(1).split(' ')
  fmarker = folium.Marker(location=[cy,cx],popup=row[7],icon=BeautifyIcon(icon_shape='circle-dot', border_color='red', border_width=10))
  # Call the callback function whenever the location of the marker changes
  fmarker.add_to(m)
#m.save(f'{path}anomalies-map.html')
m

In [ ]:
#Get anomalyId from input
from IPython.display import display, Markdown
import ipywidgets as widgets
anomalyID_select = widgets.Dropdown(
    options=df['gml:id'],
    value=df['gml:id'][0],
    description='anomalyID:')
display(Markdown('## Select one AnomalyID for 3D.'))
display(anomalyID_select)

In [ ]:
from PIL import Image
import requests
import matplotlib.pyplot as plt
from zipfile import ZipFile, BadZipFile
from io import BytesIO
import urllib
import os

anomalyId =anomalyID_select.value.split('.')[1]
anomalyJpegUrl = f'https://remanentanomalies.csiro.au/getJpeg.ashx?anomalyId={anomalyId}'
print (anomalyJpegUrl)
anomaly3dModelUrl = f'https://remanentanomalies.csiro.au/getAllModelsForAnomaly.ashx?anomalyid={anomalyId}'
print (anomaly3dModelUrl)

#download anomalyJpeg
anomalyJpeg = Image.open(requests.get(anomalyJpegUrl, stream=True).raw)
plt.imshow(anomalyJpeg)
anomalyJpeg.save(f'magnetic-anomaly-{anomalyId}.jpg')

#download anomaly3dModelUrl
try:
    z = ZipFile(BytesIO(urllib.request.urlopen(anomaly3dModelUrl).read()))
except BadZipFile:
    raise SystemExit(f'Could not download the TS model for anomaly.{anomalyId}.')
        
anomalyModelFileName:str = ''
for name in z.namelist():
    if '.zip' not in name:
        continue
    z2 = ZipFile(BytesIO(z.read(name)))
    for name2 in z2.namelist():
        if '.ts' not in name2:
            continue
        tsFile = z2.read(name2)
        anomalyModelFileName = os.path.basename(name2)
        with open(f'{anomalyModelFileName}',mode='wb') as f:
            f.write(tsFile)

In [ ]:
#Render the TS file by Trimesh
try:
    v, t, meta = read_ts(tsFile.decode("utf-8").splitlines())
    color = [255, 125, 125] 

    tri_index = t
    faces = np.empty_like(tri_index, shape=(tri_index.shape[0], 4) )
    faces[:, 0] = 3
    faces[:, 1:] = tri_index    
except Exception as gocadE:
    print(gocadE)   
#t = t -1
mesh = trimesh.Trimesh(vertices=v, faces=t-1, face_colors= color, process=False)
mesh.show()

In [ ]:
#Render the TS file by plt
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(mesh.vertices[:, 0], mesh.vertices[:,1], triangles=mesh.faces, Z=mesh.vertices[:,2]) 
plt.show()